# Challenge 2 Critique de Films

https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&sort=num_votes,asc&view=advanced

## Structure de l'URL

### Training

In [1]:
# conda install -c anaconda beautifulsoup4 à faire avant dans la console
import requests
# Librairie BeautifulSoup du package bs4
from bs4 import BeautifulSoup

In [2]:
#Solution Bruno
# Télécharger la page
url = "https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&sort=num_votes,desc"
response = requests.get(url)
content = response.content
parser = BeautifulSoup(content, 'html.parser')

print(parser.text[:5])

In [3]:
#Solution Rod
# Télécharger la page
from requests import get
url = 'https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&sort=num_votes,desc&ref_=adv_prv'
response = requests.get(url)
print(response.text[:100])



<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/


## Structure HTML de la page

In [4]:
from bs4 import BeautifulSoup
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

bs4.BeautifulSoup

In [5]:
movie_containers = html_soup.find_all('div', class_="lister-item mode-advanced")
print(type(movie_containers))
print(len(movie_containers))

<class 'bs4.element.ResultSet'>
50


## Extraire la data pour un seul film

In [6]:
first_movie = movie_containers[0]
first_movie

<div class="lister-item mode-advanced">
<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt3315342"></div>
</div>
<div class="lister-item-image float-left">
<a href="/title/tt3315342/"> <img alt="Logan" class="loadlate" data-tconst="tt3315342" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BYzc5MTU4N2EtYTkyMi00NjdhLTg3NWEtMTY4OTEyMzJhZTAzXkEyXkFqcGdeQXVyNjc1NTYyMjg@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png" width="67"/>
</a> </div>
<div class="lister-item-content">
<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt3315342/">Logan</a>
<span class="lister-item-year text-muted unbold">(2017)</span>
</h3>
<p class="text-muted">
<span class="certificate">R</span>
<span class="ghost">|</span>
<span class="runtime">137 min</span>
<span class="ghost">|</span>
<span class="genre">
Action, Drama, Sci-Fi            </span>
<

In [7]:
# Titre du film
first_movie.div

<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt3315342"></div>
</div>

In [8]:
first_movie.a

<a href="/title/tt3315342/"> <img alt="Logan" class="loadlate" data-tconst="tt3315342" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BYzc5MTU4N2EtYTkyMi00NjdhLTg3NWEtMTY4OTEyMzJhZTAzXkEyXkFqcGdeQXVyNjc1NTYyMjg@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png" width="67"/>
</a>

In [9]:
first_movie.h3

<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt3315342/">Logan</a>
<span class="lister-item-year text-muted unbold">(2017)</span>
</h3>

In [10]:
first_movie.h3.a

<a href="/title/tt3315342/">Logan</a>

In [11]:
# Titre du film
first_name = first_movie.h3.a.text
first_name

'Logan'

In [12]:
# Année de sortie
first_year = first_movie.h3.find('span', class_="lister-item-year text-muted unbold").text
first_year

'(2017)'

In [13]:
# Note IMDB
first_imdb = float(first_movie.strong.text) #convertir en float (nombre à virgule)
first_imdb

8.1

In [14]:
type(first_imdb)

float

In [15]:
# Note Metacritic
first_metascore = int(first_movie.find('span', class_="metascore favorable").text) #convertir en un nombre entier
first_metascore

77

In [16]:
# Nombre de votes
first_votes = first_movie.find('span', attrs = {'name' : 'nv'})
first_votes

<span data-value="732595" name="nv">732,595</span>

In [17]:
type(first_votes['data-value'])

str

In [18]:
first_votes = int(first_votes['data-value'])
first_votes

732595

## Script pour scraper une seule page

In [19]:
test_movie_metascore = movie_containers[15].find('div', class_='ratings-metascore')
type(test_movie_metascore)

bs4.element.Tag

In [20]:
# On créé des listes vides pour toutes nos informations

names = []
years = []
imdb_ratings = []
metascores = []
votes = []

# On reprend notre data movie_containers pour y extraire l'information
for container in movie_containers:
    
    #Si le film a une note metacritic on l'extrait
    if container.find('div', class_='ratings-metascore') is not None:
        #Le titre du film
        name = container.h3.a.text
        names.append(name) #on ajoute chaque élément name de la boucle à la liste names
        
        #L'année de sortie
        year = container.h3.find('span', class_='lister-item-year text-muted unbold').text
        years.append(year) #on ajoute chaque élément year de la boucle à la liste years
        
        #La note IMDB
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb) #on ajoute chaque élément imdb de la boucle à la liste imdb_ratings
        
        #Metascore
        m_score = container.find('span', class_='metascore').text
        metascores.append(int(m_score)) #on ajoute chaque élément m_score de la boucle à la liste metascores
        
        #Le nombre de votes
        vote = container.find('span', attrs = {'name' : 'nv'})['data-value']
        votes.append(int(vote)) #on ajoute chaque élément vote de la boucle à la liste votes

In [21]:
print(len(names))

41


## Affichage en DataFrame avec Pandas

In [22]:
#pip install pandas à faire d'abord dans la console
import pandas as pd
test_df = pd.DataFrame({
    'movies' : names,
    'years' : years,
    'imdb' : imdb_ratings,
    'metascores' : metascores,
    'votes' : votes
    })

In [23]:
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   movies      41 non-null     object 
 1   years       41 non-null     object 
 2   imdb        41 non-null     float64
 3   metascores  41 non-null     int64  
 4   votes       41 non-null     int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.7+ KB
None


In [24]:
test_df

,movies,years,imdb,metascores,votes
0,Logan,(2017),8.1,77,732595
1,Thor: Ragnarok,(2017),7.9,74,702964
2,Guardians of the Galaxy Vol. 2,(2017),7.6,67,653293
3,Wonder Woman,(2017),7.4,76,639657
4,Dunkirk,(2017),7.8,94,634431
5,Spider-Man: Homecoming,(2017),7.4,73,630844
6,Star Wars: Episode VIII - The Last Jedi,(2017),6.9,84,613881
7,Get Out,(I) (2017),7.7,85,563151
8,Blade Runner 2049,(2017),8.0,81,543049
9,It,(I) (2017),7.3,69,518283


## Script pour toutes les pages

In [25]:
pages = [str(i) for i in range (1,5)]
pages

['1', '2', '3', '4']

In [26]:
years_url = [str(i) for i in range (2000,2018)]
years_url

['2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017']

## Contrôler le taux de requêtes envoyées

In [27]:
from time import sleep
from random import randint

for a in range(0,5):
    print('Booh')
    sleep(randint(1,4))

Booh
Booh
Booh
Booh
Booh


In [28]:
from time import time
from IPython.display import clear_output
start_time = time() #on fixe le temps de début
requests = 0 #la variable requests va compter le nombre de requêtes
for _ in range(5):
    requests +=1 #on incrémente la variable requests d'une unité à chaque passage de boucle
    sleep(randint(1,3)) #on fait une pause de 1 à 3 secondes
    elpased_time = time() - start_time #on calcule le temps écoulé depuis la première requête
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elpased_time))
    clear_output(wait = True)

Request: 5; Frequency: 0.3566691064217351 requests/s


In [29]:
from warnings import warn
warn('Attention')

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Attention
  


## Script Final

In [ ]:
# 1. Redéclarer les listes
names = []
years = []
imdb_ratings = []
metascores = []
votes = []

# 2. Préparer l'affichage des boucles
start_time = time()
requests = 0

# 3. Pour chaque année de l'intervalle 2000-2017
for year_url in years_url:
    
    # 4. Boucle pour chaque page de 1 à 4
    for page in pages:
        
        # 5. Faire une requête GET
        response = get('https://www.imdb.com/search/title/?release_date='+ year_url +'&sort=num_votes,desc&start='+ page)
        
        # 6. Pause de la boucle de 8 à 15 secondes
        sleep(randint(8,15))
        
        # 7. Afficher les informations sur les requêtes
        requests += 1
        elapsed_time = time() - start_time
        print('Request: {}; Frequency: {} request/s'.format(requests, requests/elapsed_time))
        clear_output(wait = True)
        
        # 8. Avertir si le code status est différent de 200
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, requests.status_code))
            
        # 9. Stopper la boucle si le nombre de requêtes est supérieur à 72
        if requests > 72:
            warn('Nombre de requêtes trop important')
            break
        
        # 10. Extraire le HTML avec BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')
            
        # 11. Sélectionner les 50 films de chaque page (container)
        mv_containers = page_html.find_all('div', class_='lister-item mode-advanced')
        
        # 12. Boucle pour chaque container
        for container in mv_containers:
            # 13. Si le film a un Metascore
            if container.find('div', class_='ratings-metascore') is not None:
                # 14. Scrape le titre
                name = container.h3.a.text
                names.append(name)
                
                # 15. Scrape l'année
                year = container.h3.find('span', class_='lister-item-year').text
                years.append(year)
                
                # 16. Scrape la note IMDB
                imdb = float(container.strong.text)
                imdb_ratings.append(imdb)
                
                # 17. Scrape le Metascore
                metascore = container.find('span', class_='metascore').text
                metascores.append(int(metascore))
                
                # 18. Scrape le nombre de votes
                vote = container.find('span', attrs = {'name':'nv'})['data-value']
                votes.append(int(vote))

Request: 7; Frequency: 0.0831853796008851 request/s


In [ ]:
names